### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

In [3]:
keywords = ["CacheNodeidentityInfo", "EnableSeparateConfigBase", "ExperimentalClusterDNS",
            "GoogleCloudBucketACL", "KeepLaunchConfigurations", "SpecOverrideFlag", "Spotinst",
            "SpotinstOcean", "SpotinstHybrid", "SpotinstController", "VFSVaultSupport",
            "VPCSkipEnableDNSSupport", "SkipEtcdVersionCheck", "TerraformJSON", "ClusterAddons",
            "KopsControllerStateStore", "APIServerNodes", "UseAddonOperators", "AWSIPv6",
            "TerraformManagedFiles", "AlphaAllowGCE", "AlphaAllowALI"#, "Azure" 
            # removing "Azure" because it affects more than 100 files
           ] 

### List files with keywords

#### List all .go files

In [4]:
go_folders = [x[0] for x in os.walk("./kops/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./kops/vendor/google.golang.org/protobuf/internal/impl/convert.go
./kops/upup/pkg/fi/cloudup/target.go
./kops/vendor/github.com/moby/term/termios_bsd.go
./kops/vendor/sigs.k8s.io/kustomize/kyaml/internal/forked/github.com/go-yaml/yaml/apic.go
./kops/vendor/google.golang.org/appengine/internal/identity_flex.go
./kops/vendor/golang.org/x/net/ipv6/zsys_linux_ppc64le.go
./kops/vendor/k8s.io/client-go/applyconfigurations/core/v1/ephemeralcontainercommon.go
./kops/vendor/github.com/prometheus/procfs/internal/util/parse.go
./kops/upup/pkg/fi/cloudup/openstack/volume.go
./kops/vendor/k8s.io/legacy-cloud-providers/aws/aws_loadbalancer.go


In [5]:
len(go_files)

8304

#### List the interesting files, containing at least one keyword with the featureflag 

In [6]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "featureflag" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./kops/upup/pkg/fi/cloudup/apply_cluster.go contains :
-> Spotinst
-> SpotinstHybrid
-> KopsControllerStateStore
-> TerraformManagedFiles
-> AlphaAllowGCE
-> AlphaAllowALI


./kops/upup/pkg/fi/cloudup/new_cluster.go contains :
-> Spotinst


./kops/upup/pkg/fi/cloudup/template_functions.go contains :
-> CacheNodeidentityInfo
-> Spotinst
-> APIServerNodes


./kops/upup/pkg/fi/cloudup/populate_instancegroup_spec.go contains :
-> APIServerNodes


./kops/upup/pkg/fi/cloudup/terraform/target.go contains :
-> TerraformJSON
-> TerraformManagedFiles


./kops/upup/pkg/fi/cloudup/terraform/target_hcl2.go contains :
-> Spotinst


./kops/upup/pkg/fi/cloudup/bootstrapchannelbuilder/bootstrapchannelbuilder.go contains :
-> Spotinst
-> SpotinstController
-> ClusterAddons
-> UseAddonOperators


./kops/upup/pkg/fi/cloudup/alitasks/launchconfiguration.go contains :
-> KeepLaunchConfigurations


./kops/upup/pkg/fi/cloudup/awstasks/vpc.go contains :
-> VPCSkipEnableDNSSupport


./kops/upup/pkg/fi/cloudup/a

In [7]:
len(go_file_interests)

29

## Step 3 - Search Feature Toggles dependencies in the AST

In [8]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./kops/upup/pkg/fi/cloudup/apply_cluster.go 



if !featureflag.AlphaAllowGCE.Enabled() {
				return fmt.Errorf("GCE support is currently alpha, and is feature-gated.  export KOPS_FEATURE_FLAGS=AlphaAllowGCE")
			}


-> !featureflag.AlphaAllowGCE.Enabled()


if !featureflag.AlphaAllowALI.Enabled() {
				return fmt.Errorf("aliyun support is currently alpha, and is feature-gated.  export KOPS_FEATURE_FLAGS=AlphaAllowALI")
			}


-> !featureflag.AlphaAllowALI.Enabled()


if featureflag.Spotinst.Enabled() {
				l.Builders = append(l.Builders, &awsmodel.SpotInstanceGroupModelBuilder{
					AWSModelContext:        awsModelContext,
					BootstrapScriptBuilder: bootstrapScriptBuilder,
					Lifecycle:              clusterLifecycle,
					SecurityLifecycle:      securityLifecycle,
				})

				if featureflag.SpotinstHybrid.Enabled() {
					l.Builders = append(l.Builders, awsModelBuilder)
				}
			} else {
				l.Builders = append(l.Builders, awsModelBuilder)
			}


-> featureflag.



if raw == nil {
		c := &awsCloudImplementation{
			region: region,
			regionDelayers: &RegionDelayers{
				delayerMap: make(map[string]*k8s_aws.CrossRequestRetryDelay),
			},
			instanceTypes: &instanceTypes{
				typeMap: make(map[string]*ec2.InstanceTypeInfo),
			},
		}

		config := aws.NewConfig().WithRegion(region)

		// This avoids a confusing error message when we fail to get credentials
		// e.g. https://github.com/kubernetes/kops/issues/605
		config = config.WithCredentialsChainVerboseErrors(true)
		config = request.WithRetryer(config, newLoggingRetryer(ClientMaxRetries))

		// We have the updated aws sdk from 1.9, but don't have https://github.com/kubernetes/kubernetes/pull/55307
		// Set the SleepDelay function to work around this
		// TODO: Remove once we update to k8s >= 1.9 (or a version of the retry delayer than includes this)
		config.SleepDelay = func(d time.Duration) {
			klog.V(6).Infof("aws request sleeping for %v", d)
			time.Sleep(d)
		}

		requestLogger := newReq



if strings.HasPrefix(c.Spec.SecretStore, "vault://") {
		if !featureflag.VFSVaultSupport.Enabled() {
			allErrs = append(allErrs, field.Forbidden(fieldSpec.Child("secretStore"), "vault VFS is an experimental feature; set `export KOPS_FEATURE_FLAGS=VFSVaultSupport`"))
		}
		if kops.CloudProviderID(c.Spec.CloudProvider) != kops.CloudProviderAWS {
			allErrs = append(allErrs, field.Forbidden(fieldSpec.Child("secretStore"), "Vault secret store is only available on AWS"))
		}
	}


-> strings.HasPrefix(c.Spec.SecretStore, "vault://")


if !featureflag.VFSVaultSupport.Enabled() {
			allErrs = append(allErrs, field.Forbidden(fieldSpec.Child("secretStore"), "vault VFS is an experimental feature; set `export KOPS_FEATURE_FLAGS=VFSVaultSupport`"))
		}


-> !featureflag.VFSVaultSupport.Enabled()


if strings.HasPrefix(c.Spec.KeyStore, "vault://") {
		if !featureflag.VFSVaultSupport.Enabled() {
			allErrs = append(allErrs, field.Forbidden(fieldSpec.Child("keyStore"), "vault VFS is an experimental



File :             ./kops/cmd/kops/create_cluster.go 



if featureflag.EnableSeparateConfigBase.Enabled() {
		cmd.Flags().StringVar(&options.ConfigBase, "config-base", options.ConfigBase, "A cluster-readable location where we mirror configuration information, separate from the state store.  Allows for a state store that is not accessible from the cluster.")
		cmd.RegisterFlagCompletionFunc("config-base", func(cmd *cobra.Command, args []string, toComplete string) ([]string, cobra.ShellCompDirective) {
			// TODO complete vfs paths
			return nil, cobra.ShellCompDirectiveNoFileComp
		})
	}


-> featureflag.EnableSeparateConfigBase.Enabled()


if featureflag.ClusterAddons.Enabled() {
		cmd.Flags().StringSliceVar(&options.AddonPaths, "add", options.AddonPaths, "Paths to addons we should add to the cluster")
		// TODO complete VFS paths
	}


-> featureflag.ClusterAddons.Enabled()


if featureflag.AWSIPv6.Enabled() {
		cmd.Flags().BoolVar(&options.IPv6, "ipv6", false, "Allocate IPv6 CIDRs 